In [2]:
# 웹 페이지의 내용을 분석하여 csv 파일로 저장
import csv
from urllib.request import urlopen
from bs4 import BeautifulSoup

html=urlopen('http://en.wikipedia.org/wiki/Comparison_of_text_editors')
bs=BeautifulSoup(html, 'html.parser')
table=bs.find_all('table',{'class':'wikitable'})[0]
rows=table.find_all('tr')

csvFile=open('d:/data/csv/editors.csv','w',newline='',encoding='utf-8')
writer=csv.writer(csvFile)
try:
    for row in rows:
        csvRow=[]
        for cell in row.find_all(['td','th']):
            csvRow.append(cell.text.strip())
        writer.writerow(csvRow)
finally:
    print('csv로 저장되었습니다.')
    csvFile.close()

csv로 저장되었습니다.


In [5]:
from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
import re
import datetime
import random
import pymysql

conn=pymysql.connect(host='localhost', port=3306, user='web', password='1234', db='myweb', charset='utf8')
cursor=conn.cursor()
cursor.execute('use myweb')
cursor.execute('delete from pages')
random.seed(datetime.datetime.now())

def store(title, content):
    title=title.replace("'","''")
    content=content.replace("'","''")
    sql=f"insert into pages (title,content) values ('{title}','{content}')"
    cursor.execute(sql)
    conn.commit()

def getLinks(url):
    html=urlopen('http://ko.wikipedia.org'+url)
    obj=bs(html,'html.parser')
    title=obj.find('h1').text
    content=obj.find('div',{'id':'mw-content-text'}).find('p').text
    store(title,content)
    return obj.find('div', {'id':'bodyContent'}).find_all('a',href=re.compile("^(/wiki/)((?!:).)*$"))

links=getLinks('/wiki/SARS-CoV-2_%EC%98%A4%EB%AF%B8%ED%81%AC%EB%A1%A0_%EB%B3%80%EC%9D%B4')
try:
    cnt=0
    while len(links) > 0:
        cnt+=1
        if cnt >= 10:
            break
        newArticle=links[random.randint(0, len(links)-1)].attrs['href']
        print(newArticle)
        links=getLinks(newArticle)
finally:
    cursor.close()
    conn.close()
    print('완료되었습니다.')
    

/wiki/%EB%AF%B8%EA%B5%AD%EC%9D%98_%EC%BD%94%EB%A1%9C%EB%82%9819_%EB%B2%94%EC%9C%A0%ED%96%89
/wiki/%EC%BF%A0%EC%9B%A8%EC%9D%B4%ED%8A%B8%EC%9D%98_%EC%BD%94%EB%A1%9C%EB%82%9819_%EB%B2%94%EC%9C%A0%ED%96%89
/wiki/2020%EB%85%84_%EC%A3%BC%EA%B0%80_%EB%8C%80%ED%8F%AD%EB%9D%BD
/wiki/%EC%A5%90%EC%8A%A4%ED%83%B1_%ED%8A%B8%EB%A4%BC%EB%8F%84
/wiki/%EA%B8%B0%EC%8B%9C%EB%8B%A4_%ED%9B%84%EB%AF%B8%EC%98%A4
/wiki/%ED%9E%88%EB%A1%9C%EC%8B%9C%EB%A7%88%ED%98%84_%EC%A0%9C1%EA%B5%AC
/wiki/%EC%95%84%EC%98%A4%EB%AA%A8%EB%A6%AC%ED%98%84_%EC%A0%9C1%EA%B5%AC
/wiki/%ED%9B%84%EC%BF%A0%EC%98%A4%EC%B9%B4%ED%98%84_%EC%A0%9C3%EA%B5%AC
/wiki/%EB%8F%84%EC%BF%84%EB%8F%84_%EC%A0%9C21%EA%B5%AC
/wiki/%ED%9A%A8%EA%B3%A0%ED%98%84_%EC%A0%9C2%EA%B5%AC
완료되었습니다.
